# Database Testing Notebook
This notebook tests the database setup and basic operations.

In [1]:
# Test 1: Check if database file exists
import os
import sqlite3

db_path = "chatbot.db"

if os.path.exists(db_path):
    print(f"✅ Database file exists: {db_path}")
    print(f"📊 File size: {os.path.getsize(db_path)} bytes")
else:
    print(f"❌ Database file not found: {db_path}")

✅ Database file exists: chatbot.db
📊 File size: 32768 bytes


In [2]:
# Test 2: Check database tables
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print(f"📋 Tables found: {[table[0] for table in tables]}")

# Check table schemas
for table in tables:
    table_name = table[0]
    print(f"\n🔍 Schema for {table_name}:")
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    for col in columns:
        print(f"   - {col[1]} ({col[2]})")

conn.close()

📋 Tables found: ['users', 'chat_logs']

🔍 Schema for users:
   - id (INTEGER)
   - username (VARCHAR)
   - email (VARCHAR)
   - token_limit (INTEGER)
   - tokens_used (INTEGER)
   - is_active (BOOLEAN)
   - created_at (DATETIME)

🔍 Schema for chat_logs:
   - id (INTEGER)
   - user_id (INTEGER)
   - query (TEXT)
   - response (TEXT)
   - tokens_used (INTEGER)
   - timestamp (DATETIME)


In [3]:
# Test 3: Check users data
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

try:
    cursor.execute("SELECT COUNT(*) FROM users;")
    user_count = cursor.fetchone()[0]
    print(f"👥 Total users in database: {user_count}")
    
    cursor.execute("SELECT username, email, token_limit, tokens_used, is_active FROM users;")
    users = cursor.fetchall()
    
    print("\n📝 User details:")
    for user in users:
        status = "Active" if user[4] else "Inactive"
        print(f"   - {user[0]} ({user[1]})")
        print(f"     Token limit: {user[2]}, Used: {user[3]}, Status: {status}")
        
except Exception as e:
    print(f"❌ Error checking users: {e}")

conn.close()

👥 Total users in database: 3

📝 User details:
   - user1 (user1@company.com)
     Token limit: 1000, Used: 0, Status: Active
   - user2 (user2@company.com)
     Token limit: 1000, Used: 0, Status: Active
   - demo (demo@company.com)
     Token limit: 500, Used: 0, Status: Active


In [4]:
# Test 4: Test adding a chat log (simulate usage)
import sys
import os

# Add current directory to path
current_dir = os.getcwd()
sys.path.insert(0, current_dir)

from datetime import datetime

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

try:
    # Add a test chat log
    test_log = {
        'user_id': 1,  # user1
        'query': 'What is our company policy?',
        'response': 'Our company policy covers various aspects including security guidelines and remote work options.',
        'tokens_used': 25,
        'timestamp': datetime.now().isoformat()
    }
    
    cursor.execute("""
        INSERT INTO chat_logs (user_id, query, response, tokens_used, timestamp)
        VALUES (?, ?, ?, ?, ?)
    """, (test_log['user_id'], test_log['query'], test_log['response'], 
          test_log['tokens_used'], test_log['timestamp']))
    
    conn.commit()
    print("✅ Test chat log added successfully!")
    
    # Check chat logs
    cursor.execute("SELECT COUNT(*) FROM chat_logs;")
    log_count = cursor.fetchone()[0]
    print(f"💬 Total chat logs: {log_count}")
    
except Exception as e:
    print(f"❌ Error adding chat log: {e}")

conn.close()

✅ Test chat log added successfully!
💬 Total chat logs: 1


In [5]:
# Test 5: Test config file
try:
    import config
    print("✅ Config file loaded successfully!")
    print(f"📊 Database URL: {config.DATABASE_URL}")
    print(f"🔑 OpenAI API Key: {'Set' if config.OPENAI_API_KEY != 'your-api-key-here' else 'Not set'}")
    print(f"👤 Admin Username: {config.ADMIN_USERNAME}")
    print(f"🎯 Default Token Limit: {config.DEFAULT_TOKEN_LIMIT}")
except Exception as e:
    print(f"❌ Error loading config: {e}")
    print("Please make sure config.py exists and is properly configured.")

✅ Config file loaded successfully!
📊 Database URL: sqlite:///./chatbot.db
🔑 OpenAI API Key: Set
👤 Admin Username: admin_vulcan
🎯 Default Token Limit: 1000
